# Bi-LSTM for Feature Prediction

In [ ]:
# imports
import pandas as pd
import numpy as np
import wandb

In [ ]:
from models.bilstm import BiLSTMDatasetManager, BiLSTMModelManager

In [ ]:
data_path = "../data/processed/features_pca_iv16-20.csv"

In [ ]:
dataset = BiLSTMDatasetManager(data_path)
features, targets = dataset.make_train_target_pairs()
print('Features shape:', features.shape)
print('Targets shape:', targets.shape)

In [ ]:
print('Initializing BiLSTM model...')
model_path = './ckpts/test_bilstm256.pth'
model = BiLSTMModelManager(input_dim=9, hidden_dim=256, output_dim=3, learning_rate=0.01, model_path=model_path)
model.train(features, targets, epochs=250000)

## DNN

In [1]:
import pandas as pd
import numpy as np
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
from models.bilstm import CustomBiLSTMModel

In [2]:
model_path = './ckpts/test_bilstm256.pth'
bilstm_model = CustomBiLSTMModel(input_dim=9, hidden_dim=256, output_dim=3)
bilstm_model.load_state_dict(torch.load(model_path))

/tmp/ipykernel_42272/4007657262.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  bilstm_model.load_state_dict(torch.load(model_path))


<All keys matched successfully>

In [3]:
features = pd.read_csv("../data/processed/features_pca_iv16-20.csv")
features.head()

,date,feature1,feature2,feature3
0,2016-01-05,-2.946036,0.399305,2.333101
1,2016-01-06,-2.890993,0.153763,2.388737
2,2016-01-07,-4.476846,2.506460,2.701483
3,2016-01-08,-3.925725,1.810529,2.540009
4,2016-01-11,-4.277851,2.453446,2.562114


In [4]:
for i in range(22,len(features)):
    ma1 = torch.tensor(features.iloc[i-1][['feature1', 'feature2', 'feature3']].astype(float).values, dtype=torch.float32)
    ma5 = torch.tensor(features.iloc[i-5:i][['feature1', 'feature2', 'feature3']].mean(axis=0).values, dtype=torch.float32)
    ma22 = torch.tensor(features.iloc[i-22:i][['feature1', 'feature2', 'feature3']].mean(axis=0).values, dtype=torch.float32)
    feature = torch.cat((ma1, ma5, ma22), dim=0).to(device)
    out = bilstm_model.predict(feature)
    for obj in out:
        features.at[i, "F1"] = obj[0].item()
        features.at[i, "F2"] = obj[1].item()
        features.at[i, "F3"] = obj[2].item()

In [5]:
features = features.dropna().reset_index(drop=True)
features.head()

,date,feature1,feature2,feature3,F1,F2,F3
0,2016-02-05,-2.785850,0.208685,2.273442,-2.866897,0.122800,2.436066
1,2016-02-08,-2.426245,-0.018148,2.072642,-2.711939,0.211495,2.214754
2,2016-02-09,-2.978782,0.438694,2.333295,-3.057659,0.473622,2.400317
3,2016-02-10,-3.273419,0.806418,2.418158,-3.468703,1.069636,2.478444
4,2016-02-11,-2.607269,0.123359,2.159899,-2.746451,0.177474,2.305250


In [6]:
df_iv_path_list = [
    "../data/processed/pca/predicted_iv16.csv",
    "../data/processed/pca/predicted_iv17.csv",
    "../data/processed/pca/predicted_iv18.csv",
    "../data/processed/pca/predicted_iv19.csv",
    "../data/processed/pca/predicted_iv20.csv"
]

In [7]:
merged_df = pd.DataFrame()
for path in df_iv_path_list:
    df = pd.read_csv(path)
    merged_df = pd.concat([merged_df, df], axis=0)

merged_df = merged_df.reset_index(drop=True)
print(len(merged_df))
merged_df.head()

193424


,date,tau,m,IV
0,2016-01-04,0.027397,-0.510826,0.326153
1,2016-01-04,0.027397,-0.223144,0.291228
2,2016-01-04,0.027397,-0.105361,0.286565
3,2016-01-04,0.027397,-0.051293,0.286299
4,2016-01-04,0.027397,-0.025318,0.286591


In [8]:
# join the two dataframes using the date column so that we have the corresponding F1, F2, F3 values for each date
df = pd.merge(merged_df, features, on='date')
df.head()

,date,tau,m,IV,feature1,feature2,feature3,F1,F2,F3
0,2016-02-05,0.027397,-0.510826,0.346719,-2.78585,0.208685,2.273442,-2.866897,0.1228,2.436066
1,2016-02-05,0.027397,-0.223144,0.309534,-2.78585,0.208685,2.273442,-2.866897,0.1228,2.436066
2,2016-02-05,0.027397,-0.105361,0.304773,-2.78585,0.208685,2.273442,-2.866897,0.1228,2.436066
3,2016-02-05,0.027397,-0.051293,0.304623,-2.78585,0.208685,2.273442,-2.866897,0.1228,2.436066
4,2016-02-05,0.027397,-0.025318,0.305006,-2.78585,0.208685,2.273442,-2.866897,0.1228,2.436066


In [9]:
len(df)

189882

In [11]:
df_train = df[:10000]
df_val = df[-2000:]

In [12]:
feature_cols = ['F1', 'F2', 'F3']
from models.dnn import IVDataset, IVSDNN, train_model, large_moneyness_penalty, butterfly_arbitrage_penalty, calendar_spread_penalty, safe_divide

In [13]:
dataset_train = IVDataset(df_train, feature_cols)
dataset_val = IVDataset(df_val, feature_cols)


Tensor shapes:
Features: torch.Size([10000, 3])
m: torch.Size([10000, 1])
tau: torch.Size([10000, 1])
iv: torch.Size([10000, 1])

Checking for NaN values:
Features NaN: False
m NaN: False
tau NaN: False
iv NaN: False

Tensor shapes:
Features: torch.Size([2000, 3])
m: torch.Size([2000, 1])
tau: torch.Size([2000, 1])
iv: torch.Size([2000, 1])

Checking for NaN values:
Features NaN: False
m NaN: False
tau NaN: False
iv NaN: False


In [14]:
print(dataset_train.get_input_size())
print(dataset_val.get_input_size())

5
5


In [15]:
from torch.utils.data import DataLoader
train_loader = DataLoader(dataset_train, batch_size=64, shuffle=True)
val_loader = DataLoader(dataset_val, batch_size=64, shuffle=False)
dnn = IVSDNN(input_size=dataset_train.get_input_size(), hidden_size=512)

In [16]:
import wandb
wandb.init(project="ivs-dnn")
train_model(dnn, train_loader, val_loader, 200, 0.001, 1, wandb)

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: abhigyanshanker (abx-group). Use `wandb login --relogin` to force relogin


/home/shankerabhigyan/miniconda3/envs/torch/lib/python3.11/site-packages/torch/optim/lr_scheduler.py:60: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(
Epoch 1/200 - Validation: 100%|██████████| 32/32 [00:34<00:00,  1.07s/it]



Epoch 1 Results:
Training || Loss: 5467199.631508 | MSE: 0.063333 | MAPE: 99.532249 | Penalty: 27335995.961902
Validation || Loss: 0.431532 | MSE: 0.078837 | MAPE: 99.999380 | Penalty: 0.000000
Learning Rate: 0.001000


Epoch 2/200 - Validation: 100%|██████████| 32/32 [00:33<00:00,  1.05s/it]



Epoch 2 Results:
Training || Loss: 0.423771 | MSE: 0.059184 | MAPE: 99.995059 | Penalty: 0.000585
Validation || Loss: 0.439814 | MSE: 0.078821 | MAPE: 99.984962 | Penalty: 0.041727
Learning Rate: 0.001000


...


Epoch 17 Results:
Training || Loss: 0.058411 | MSE: 0.003327 | MAPE: 14.247517 | Penalty: 0.000452
Validation || Loss: 1245.095800 | MSE: 0.078837 | MAPE: 100.000000 | Penalty: 6223.321190
Learning Rate: 0.000090


Epoch 18/200 - Training:  93%|█████████▎| 146/157 [08:04<00:36,  3.32s/it]


KeyboardInterrupt: 

Error in callback <bound method _WandbInit._pause_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7972869b1b50>> (for post_run_cell), with arguments args (<ExecutionResult object at 7972869b3510, execution_count=16 error_before_exec=None error_in_exec= info=<ExecutionInfo object at 797293521510, raw_cell="import wandb
wandb.init(project="ivs-dnn")
train_m.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell:/home/shankerabhigyan/code/thesis/thesis-IVS/baseline/pipeline_bilstm.ipynb#X31sZmlsZQ%3D%3D> result=None>,),kwargs {}:


BrokenPipeError: [Errno 32] Broken pipe